# Web Scraping Data from PeoplePerHour

## Overview  
This script scrapes freelancer data from [PeoplePerHour](https://www.peopleperhour.com/hire-freelancers),  
collecting multiple pages in a single run.

---

## **XPath Expressions**
| Data Field    | XPath Expression |
|--------------|----------------|
| **Freelancer Name** | `//h2[contains(@class, 'clearfix')]` |
| **Job Title / Description** | `//p[contains(@class, 'job-title')]` |
| **Nationality** | `//div/span[contains(@class, 'small')]` |
| **Price** | `//div/span[contains(@class, 'title-nano card')]` |
| **Rating** | `//div[contains(@class, 'ratings')]` |
| **Next Page Button** | `//li[contains(@class, 'pagination__item--next')]/a` |

---

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

# Initialize WebDriver
options = Options()
options.add_argument("--headless")  # Run in headless mode (optional)
driver = webdriver.Chrome()

# Open the website
driver.get("https://www.peopleperhour.com/hire-freelancers")
time.sleep(3)  # Allow time for elements to load

data = []
max_rows = 1
collected_rows = 0
while collected_rows < max_rows:
    freelancers = driver.find_elements(By.XPATH, "//h2[contains(@class, 'clearfix')]")
    descriptions = driver.find_elements(By.XPATH, "//p[contains(@class, 'job-title')]")
    nationalities = driver.find_elements(By.XPATH, "//div/span[contains(@class, 'small')]")
    prices = driver.find_elements(By.XPATH, "//div/span[contains(@class, 'title-nano card')]")
    ratings_data = driver.find_elements(By.XPATH, "//div[contains(@class, 'ratings')]")
    
    for i in range(len(freelancers)):
        if collected_rows >= max_rows:
            break
        
        name = freelancers[i].text if i < len(freelancers) else "N/A"
        description = descriptions[i].text if i < len(descriptions) else "N/A"
        nationality = nationalities[i].text if i < len(nationalities) else "N/A"
        price = prices[i].text if i < len(prices) else "N/A"
        
        rating_text = ratings_data[i].text if i < len(ratings_data) else "N/A"
        rating_parts = rating_text.split(" ")
        rating = rating_parts[0] if len(rating_parts) > 0 else "N/A"
        total_votes = rating_parts[1][1:-1] if len(rating_parts) > 1 else "N/A"  # Removing parentheses
        
        data.append([name, description, nationality, price, rating, total_votes])
        collected_rows += 1

    # Try to go to the next page
        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//li[contains(@class, 'pagination__item--next')]/a"))
            )
            driver.execute_script("arguments[0].scrollIntoView();", next_button)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", next_button)
            print(f"Moved to page {i + 1}")
            time.sleep(3)
        except:
            print(f"Next button not found on page {i}, stopping.")
            break

# Close the driver
driver.quit()

# Save to DataFrame and CSV
columns = ["Name", "Description", "Nationality", "Price", "Rating", "Total Votes"]
df = pd.DataFrame(data, columns=columns)
df.to_csv("freelancers_data.csv", index=False)

print("Scraping completed. Data saved to freelancers_data.csv.")

Moved to page 1
Scraping completed. Data saved to freelancers_data.csv.


In [2]:
df

,Name,Description,Nationality,Price,Rating,Total Votes
0,Tom M.,Google Ads Partner / PPC / AdWords / Google Sh...,United Kingdom,$51/hr,5.0\n,128)\nCER
1,Maria H.,"Experienced Team of Graphic Designers, Web Dev...",United Kingdom,$32/hr,4.9\n,7123)\nTO
2,Ann-Marie M.,Freelance Paralegal,Jamaica,$19/hr,,N/A
3,Maykal P.,Full stack Web / App developer / AI profession...,Bulgaria,$30/hr,5.0\n,19)\nCER
4,White Hat SEO Guru| Guaranteed Ranking S.,PPH's #1 for SEO & 22 Years of Excellence |Goo...,India,$10/hr,4.8\n,4776)\nTO
5,Green D.,"SEO Expert, 5000+ Reviews, #1in SEO & Marketin...",Ireland,$45/hr,5.0\n,6184)\nTO
6,Sunday E.,Web designer,United Kingdom,$19/hr,5.0\n,1)\nCER
7,GAJURA C.,SEO Expert and Experienced Guest Post Writer o...,Germany,$32/hr,5.0\n,2498)\nTO
8,Out of Box Studios,"UK's Leading Digital Design, Development & Mar...",United Kingdom,$38/hr,4.9\n,1677)\nTO
9,Writing Expertise,⭐ Copywriter | Proofreader| SEO Specialist |Mu...,United Kingdom,$14/hr,5.0\n,8)\nTO
